<a href="https://colab.research.google.com/github/Tavo826/DataScience/blob/main/DriverDrowsinessDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

El objetivo es detectar si la persona cierra los ojos por unos segundos

### Pasos

1. Tomar imágenes de la cámara como entradas

2. Detectar el rostro y crear una región de interés (ROI)

3. Detectar los ojos y alimentar el clasificador

4. El clasificador detecta si los ojos están abiertos o cerrados

5. Se calcula el puntaje para verificar si la persona está somnolienta

### Dataset

El dataset es creado capturando imágenes de los ojos y guardándolas en el sistema. Se separan en carpetas 'Open' o 'Closed'. Se debe limpiar el conjunto de datos manualmente, removiendo las imágenes que no se quieren para entrenar el modelo. El conjunto de datos contiene larededor de 7000 imágenes de ojos de personas bajo diferentes condicione de luz

### Arquitectura

Se crea con keras una red neuronal convolucional (CNN) con las siguientes capas:

* Capa convolucional, 32 nodos, kernel 3
* Capa convolucional, 32 nodos, kernel 3
* Capa convolucional, 64 nodos, kernel 3
* Capa densa, 128 nodos
* Capa densa, 2 nodos

Se usa activación relu para todas las capas, excepto en la de salida que usa softma

In [ ]:
!pip install pygame

     |████████████████████████████████| 11.8MB 5.1MB/s 


In [ ]:
import os
import cv2 #(no funciona en colab)
import random
import shutil
import numpy as np
import matplotlib.pyplot as plt

from keras.preprocessing import image
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dropout, Conv2D, Flatten, Dense, MaxPooling2D, BatchNormalization
#Alarma para activar cuando se detecta somnolencia
from pygame import mixer


os.chdir('/content/drive/MyDrive/Colab Notebooks/Data Science/Detección de somnolencia del conductor')

pygame 2.0.1 (SDL 2.0.14, Python 3.6.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


## Modelo

In [ ]:
def generator(dir, 
              gen=image.ImageDataGenerator(rescale=1./255),
              shuffle=True, batch_size=1, target_size=(24,24),
              class_mode='categorical'):
  
  return gen.flow_from_directory(dir, 
                                 batch_size=batch_size,
                                 shuffle=shuffle,
                                 color_mode='grayscale',
                                 class_mode=class_mode,
                                 target_size=target_size)
  
BS = 32
TS = (24,24)
train_batch = generator('data/train', shuffle=True, batch_size=BS, target_size=TS)
valid_batch = generator('data/valid', shuffle=True, batch_size=BS, target_size=TS)
#Pasos por época
SPE = len(train_batch.classes) // BS
#Pasos de alidación
VS = len(valid_batch.classes) // BS
print(SPE, VS)

model = Sequential([
                    Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(24,24,1)),
                    MaxPooling2D(pool_size=(1,1)),
                    Conv2D(32, (3,3), activation='relu')
                    MaxPooling2D(pool_size=(1,1)),
                    Conv2D(64, (3,3), activation='relu')
                    #Se escogen las mejores características vía pooling
                    MaxPooling2D(pool_size=(1,1)),
                    #Se apagan y encienden nodos para mejorar la convergencia
                    Dropout(0.25),
                    #Solamente se desea una clasificación a la salida
                    Flatten(),
                    #Obteniendo todos los datos relevantes
                    Dense(128, activation='relu')
                    Dropout(0.5)
                    #generar un softmax para aplastar la matriz en probabilidades de salida
                    Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_batch, validation_data=valid_batch, epochs=15,
                    steps_per_epoch=SPE, validation_steps=VS)
model.save('models/cnnCat2.h5', overwrite=True)

## Compilando

In [ ]:
from scipy.io import wavfile
#mixer.init() #no funciona en colab
#sound = mixer.Sound('alarm.wav')
sound = wavfile.read('alarm.wav')

In [ ]:
#Se cargan los modelos de reconocimiento
face = cv2.CascadeClassifier('haar cascade files/haarcascade_frontalface_alt.xml')
leye = cv2.CascadeClassifier('haar cascade files/haarcascade_lefteye_2splits.xml')
reye = cv2.CascadeClassifier('haar cascade files/haarcascade_righteye_2splits.xml')

#labels
lbl = ['Close', 'Open']

In [ ]:
#Se carga el modelo preentrenado de clasificación
model = load_model('models/cnnat2.h5')
path = os.getcwd()
cap = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_COMPLEX_SMALL
count = 0
score = 0
thicc = 2
rpred = [99]
lpred = [99]

while True:
  #Empieza a grabar la webcam
  ret, frame = cap.read()
  height, width = frame.shape[:2]
  #Escala de grises
  gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  #Detectando rostros
  faces = face.detectMultiScale(gray, minNeighbors=5, scaleFactor=1.1, minSize=(25,25))
  #Detectando ojos
  left_eye = leye.detectMultiScale(gray)
  right_eye = reye.detectMultiScale(gray)

  cv2.rectangle(frame, (0,height-50), (200, height), (0,0,0), thickness=cv2.FILLED)

  #Encerrando la región de interés (rostro)
  for (x,y,w,h) in faces:
    cv2.rectangle(frame, (x,y), (x+w,y+h), (100,100,100), 1)
  
  #Alimentando el modelo con imágenes de los ojos
  for (x,y,w,h) in right_eye:
    #Tomando el área del ojo
    r_eye = frame[y:y+h,x:x+w]
    #Contando el ojo
    count += 1
    #escala de grises
    r_eye = cv2.cvtColor(r_eye, cv2.COLOR_BGR2GRAY)
    #reescalando la imagen
    r_eye = cv2.resize(r_eye, (24,24))
    #normalizando
    r_eye = r_eye / 255
    r_eye = r_eye.reshape(24,24,-1)
    r_eye = np.expand_dims(r_eye, axis=0)
    #Prediciendo con el modelo
    rpred = model.predict_classes(r_eye)

    if (rpred[0] == 1):
      lbl = 'Open'
    if (rpred[0] == 0):
      lbl = 'Closed'

    break

  for (x,y,w,h) in left_eye:
    l_eye = frame[y:y+h,x:x+w]
    count += 1
    l_eye = cv2.cvtColor(l_eye, cv2.COLOR_BGR2GRAY)
    l_eye = cv2.resize(l_eye, (24,24))
    l_eye = l_eye / 255
    l_eye = l_eye.reshape(24,24,-1)
    l_eye = np.expand_dims(l_eye, axis=0)
    lpred = model.predict_classes(l_eye)

    if (lpred[0] == 1):
      lbl = 'Open'
    if (lpred[0] == 0):
      lbl = 'Closed'

    break

  #Si los ojos están cerrados
  if (rpred[0] == 0 and lpred[0] == 0):
    score += 1
    cv2.putText(frame, 'Closed', (10, height-20), font, 1, (255,255,255), 1, cv2.LINE_AA)
  #Si están abiertos
  else:
    score -= 1
    cv2.putText(frame, 'Open', (10,height-20), font, 1, (255,255,255), 1, cv2.LINE_AA)

  if (score < 0):
    score = 0
  cv2.putText(frame, 'Score: ' + str(score), (100,height-20), font, 1, (255,255,255), 1, cv2.LINE_AA)
  #más de 15 frames con los ojos cerrados
  if (score > 15):
    #Toma una captura
    cv2.imwrite(os.path.join(path, 'image.jpg'), frame)
    #Suena alarma
    try:
      sound.play() #creo??? -_-
    except:
      pass
    
    if (thicc < 16):
      thicc += 2
    else:
      thicc -= 2

    cv2.rectangle(frame, (0,0), (width, heihgt), (0,0,255), thicc)
  
  cv2.imshow('frame', frame)
  if cv2.waitKey(1) & 0xFF == ord('q'):
    break

cap.release()
cv2.destroyAllWindows()